In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

import pandas as pd
import numpy as np

import json


### 读取数据集

In [15]:

df = pd.read_csv('医疗费用预测insurance.csv')
with open("insuranceCfg.json", "r", encoding="utf-8") as f:
        cfg = json.load(f)
print("cfg:",cfg)

#编码
df['sex'] = df['sex'].map({'female':0,'male':1})  
df['smoker'] = df['smoker'].map({'no':0,'yes':1})
df['region'] = df['region'].map({'northeast':0,'northwest':1,'southeast':2,'southwest':3})


features = df.iloc[:, :-1] # 特征
features = torch.tensor(features.values,dtype=torch.float32)

labels = df.iloc[:,-1]  # 标签
labels = torch.tensor(labels,dtype=torch.float32)

class insuranceDataset(Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
batch_Size = int(cfg['batch_Size'])  #32

dataset = insuranceDataset(features, labels)
train_loader = DataLoader(dataset, batch_Size, shuffle=True)





cfg: {'batch_Size': '32', 'epochs': '100', 'lr': '0.001', 'input_size': '6', 'hidden_size': '8', 'output_size': '1'}


### 模型搭建

In [16]:
class insuranceModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(insuranceModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

### 训练

In [ ]:
for epoch in range(int(cfg['epochs'])):
    for i, (features, labels) in enumerate(train_loader):

        input_size, hidden_size, output_size = int(cfg['input_size']), int(cfg['hidden_size']), int(cfg['output_size'])

        model = insuranceModel(input_size, hidden_size, output_size)  # input_size=6, hidden_size=4, output_size=1
        
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=float(cfg['lr']))
        optimizer.zero_grad()
        
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, cfg['epochs'], i+1, len(train_loader), loss.item()))

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, torch.memory_format memory_format = None, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
